In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.rcParams["figure.figsize"] = (15,4)

In [2]:
resample = False

### Data Loading

One must have unziped files in data folder. :)

In [3]:
import os
path_raw_files = '../data'
csv_files = [path_raw_files + '/' + file for file in os.listdir(path_raw_files) if ('.csv' in file) and ('.zip' not in file)]

In [4]:
# Example of how to load times series csv file
# Each file contains data from approximately 50 devices
dfs = []
for file in tqdm(csv_files):
    print('Loading data...')
    # Sample 100 rows of data to determine dtypes.
    df_test = pd.read_csv(file, nrows=100)

    float_cols = [c for c in df_test if df_test[c].dtype == "float64"]
    float32_cols = {c: np.float32 for c in float_cols}

    df = pd.read_csv(file, engine='c', dtype=float32_cols).dropna()
    #df = pd.read_csv(file).dropna()
    dfs.append(df)
print('Finished...')

  0%|          | 0/4 [00:00<?, ?it/s]

Loading data...


 25%|██▌       | 1/4 [00:08<00:25,  8.44s/it]

Loading data...


 50%|█████     | 2/4 [00:15<00:15,  7.80s/it]

Loading data...


 75%|███████▌  | 3/4 [00:23<00:07,  7.92s/it]

Loading data...


100%|██████████| 4/4 [00:31<00:00,  7.89s/it]

Finished...


In [6]:
print('Concatenate...')
df = pd.concat(dfs, axis = 0)
print('Finished...')

Concatenate...
Finished...


In [7]:
df = df.dropna()

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

In [8]:
#power and current
df['power_out'] = df['current_out']*df['battery_voltage']
df['usb_power'] = df['usb_current']*df['battery_voltage']
df['power_in'] = df['current_in']*df['battery_voltage']
df['net_power_battery'] = df['current']*df['battery_voltage']

In [10]:
df['other_out_current'] = df['current_out'] - df['usb_current']

# finally resampling

In [11]:
if resample == True:
    df = df.groupby('battery_id').resample('10min').mean()
    df = df.drop('battery_id', axis = 1)

    df = df.reset_index().set_index('timestamp')

In [12]:
import os
if os.path.isdir('../data/cleaned_data'):
    print("Exists")
else:
    print("Doesn't exists")
    os.mkdir('../data/cleaned_data')

Doesn't exists


In [13]:
file_cleaned_name = "all_dataset"
df.to_parquet(f'../data/cleaned_data/{file_cleaned_name}.parquet')